# Bllossom-8B 양자화 모델을 이용한 한국어 LLM 튜토리얼

## 01. 활용할 package 설정
 - GPU사용하기: colab에서 런타임 --> 런타임유형변경 --> T4 선택
 - 패키지설치: 아래 pip를 이용해 Transformers, accelerate 설치
 - 런타임재시작: 런타임 --> 세션다시시작  (accelerate설치 시 런타임 다시시작하셔야됩니다!)

In [1]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./' #Bllossom모델 다운로드

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [2]:
from llama_cpp import Llama
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## 02. 모델준비

## 03. 추론- RTF

In [ ]:
PROMPT = '''Role : "당신은 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다. 제시문 세트 안의 제시문 개수와, 질문의 개수는 정해져 있지 않습니다.
당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 유사하게 새로운 제시문 세트로부터 질문을 생성하면 됩니다. 질문만 생성하는 것입니다. "

Format :
"먼저 예시로 주어진 제시문 세트입니다. 이 제시문 세트는 3개의 제시문으로 이뤄져 있습니다.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'


다음으로 예시 질문을 보여드리겠습니다. 위의 예시 제시문 세트로부터 생성된 질문입니다.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'

이처럼 예시 질문들은 예시 제시문의 내용을 참고하여, 이를 요약하거나 이들로부터 생각해볼 수 있는 내용들을 질문합니다. 이때 각 제시문 내용간의 관계를 고려해, 이를 물어보는 문제를 출제해야 합니다."

'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고 질문을 생성해주세요. [새로운 제시문]으로부터 질문을 생성하는 것입니다.
질문만 생성해주세요. 다른 불필요한 문장은 삭제해 주세요.
생성 형식은
["질문 1"
"질문 2"
"질문 3"]
으로 부탁드립니다.
질문을 3개 생성해주세요. 질문만 생성하면 됩니다. 질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다. 답변은 필요 없습니다.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에
충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이
행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을
얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는
경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상
존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을
기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한
유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이
모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은
결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고
동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를
닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과
동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는
선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상
존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에
사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀
있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인,
프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운
것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에
확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서
변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을
하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령, ‘엄마, 아빠’와 같은 말은
어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적
행동 양식이 반영된 것이라고 할 수 있다. 이런 ‘연령 단계’에 의한 언어 차이 외에도,
기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도
있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를
쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말
모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와
신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는
젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  

In [ ]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.6,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])


llama_print_timings:        load time =   10726.30 ms
llama_print_timings:      sample time =     310.83 ms /   123 runs   (    2.53 ms per token,   395.71 tokens per second)
llama_print_timings: prompt eval time =   11691.76 ms /  2035 tokens (    5.75 ms per token,   174.05 tokens per second)
llama_print_timings:        eval time =    2886.64 ms /   122 runs   (   23.66 ms per token,    42.26 tokens per second)
llama_print_timings:       total time =   15031.19 ms /  2157 tokens


[질문 1] 전통적인 가치에 대한 거부과 개인적인 자유에 대한 강조가 현대 사회의 특징이라는데 동의하느냐? [예를 들어, 유행] 

[질문 2] 유행은 누구의 주도적인 역할을 통해 형성되는지 궁금합니다. 예를 들어, 피렌체에서 남성 복장의 뚜렷한 유행이 탄생한 이유는 어떤 요인이 작용한 것일까요? 

[질문 3] 변화 속도가 빠른 현대 사회에서 유행이 가지는 의미가 어떻게 되는지 봅니다. 유행이 모든 분야에 영향을 미치는 경향이 있는 이유에 대해서 논의해주세요.


## 03. 추론- CARE

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model_care = Llama(
    model_path='./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=10000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.f

In [ ]:
prompt_care = '''
Context : "당신은 대입 논술 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다."
Action : "당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 새로운 제시문 세트로부터 당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 새로운 유사하게 제시문 세트로부터 질문을 생성하면 됩니다.질문을 생성하면 됩니다. 제시문 세트 안의 제시문 개수와, 질문의 개수는 정해져 있지 않습니다. "

Example : "예시 제시문 세트는 3개의 제시문으로 이뤄져 있습니다.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'

위의 예시 제시문 세트로부터 생성된 질문입니다.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'
"

Result : "이처럼 질문들은 제시문의 내용을 참고하여, 이를 요약하거나 이들로부터 생각해볼 수 있는 내용들을 질문합니다. 이때 각 제시문 내용간의 관계를 고려해, 이를 물어보는 문제를 출제해야 합니다. 당신은 이와 유사하게 새로운 제시문으로부터 질문을 생성하면 됩니다."
'''

In [ ]:
test_text_care = '''다음 3개의 제시문을 보고 질문을 생성해주세요.
질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다. [새로운 제시문]으로부터 질문을 생성하는 것입니다.
질문만 생성해주세요. 다른 불필요한 문장은 삭제해 주세요. 답변은 생성하지 않습니다.

생성 형식은
["질문 1"
"질문 2"
"질문 3"] 으로 해야 합니다.
질문을 3개 생성해주세요.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다.
이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다.
유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다.
그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다.
젊은 세대는 ‘반모’(반말모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
'''

In [ ]:


messages = [
    {"role": "system", "content": f"{prompt_care}"},
    {"role": "user", "content": f"{test_text_care}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":5000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.6,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model_care(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =     546.35 ms
llama_print_timings:      sample time =     199.41 ms /    62 runs   (    3.22 ms per token,   310.92 tokens per second)
llama_print_timings: prompt eval time =    1055.72 ms /   810 tokens (    1.30 ms per token,   767.25 tokens per second)
llama_print_timings:        eval time =    1779.16 ms /    61 runs   (   29.17 ms per token,    34.29 tokens per second)
llama_print_timings:       total time =    3194.70 ms /   871 tokens


질문:
1. 유행은 누구의 주도적인 역할을 통해 형성되는지 설명해주세요.
2. 어떤 요인들이 유행을 형성하는 데 중요한 역할을 하는지 설명하시오.
3. 왜 사람들은 유행에 민감하고, 새로운 변화에 쉽게 적응하는지 설명해주세요.


# 2차 시도

## 03. 추론- RTF

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  

In [ ]:
PROMPT = '''Role : "당신은 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다. 제시문 세트 안의 제시문 개수와, 질문의 개수는 정해져 있지 않습니다.
당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 유사하게 새로운 제시문 세트로부터 질문을 생성하면 됩니다. 질문만 생성하는 것입니다. "

Format :
"먼저 예시로 주어진 제시문 세트입니다. 이 제시문 세트는 3개의 제시문으로 이뤄져 있습니다.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'


다음으로 예시 질문을 보여드리겠습니다. 위의 예시 제시문 세트로부터 생성된 질문입니다.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'

이처럼 예시 제시문들은 공통적인 주제를 가지고, 예시 질문들은 예시 제시문의 내용을 참고해 제시문 간의 내용의 연관성을 묻습니다. 당신은 이와 유사하게 새로운 제시문 세트로부터 문제를 출제해야 합니다."

'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고 질문을 생성해주세요. [새로운 제시문]으로부터 질문을 생성하는 것입니다.
생성 형식은
["질문 1"
"질문 2"
"질문 3"]
으로 부탁드립니다.

질문을 3개 생성해주세요. 질문만 생성하면 되고 답변은 필요 없습니다. 다른 불필요한 문장은 삭제해 주세요. 질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에
충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이
행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을
얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는
경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상
존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을
기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한
유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이
모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은
결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고
동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를
닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과
동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는
선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상
존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에
사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀
있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인,
프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운
것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에
확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서
변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을
하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령, ‘엄마, 아빠’와 같은 말은
어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적
행동 양식이 반영된 것이라고 할 수 있다. 이런 ‘연령 단계’에 의한 언어 차이 외에도,
기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도
있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를
쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말
모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와
신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는
젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.6,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?

llama_print_timings:        load time =     546.52 ms
llama_print_timings:      sample time =     506.82 ms /   192 runs   (    2.64 ms per token,   378.84 tokens per second)
llama_print_timings: prompt eval time =    2160.73 ms /  2029 tokens (    1.06 ms per token,   939.03 tokens per second)
llama_print_timings:        eval time =    5576.66 ms /   191 runs   (   29.20 ms per token,    34.25 tokens per second)
llama_print_timings:       total time =    8720.92 ms /  2220 tokens


[질문 1] 유행이 모든 세대에 영향을 미치는가요?
유행은 시대에 따라 영향을 주기도 하지만, 어떤 시대에는 유행이 지배적일지라도 급격한 변화를 겪을 수 있습니다.

[질문 2] 어떤 요인들이 유행을 이끌어내는 역할을 합니다?
유행은 크게 남성복장의 경우, 여성복장의 경우, 그리고 디자인의 경우 세 가지 유형으로 나뉩니다. 이 중 어느 것이 가장 큰 영향을 끼치느냐는 시대에 따라 다를 수 있습니다.

[질문 3] 유행이 바뀌면 어떤 결과가 나타나는가요?
유행이 바뀌면 많은 사람들의 행동과 의식이 함께 변화합니다. 예를 들어, 유행이 바뀌면 사람들은 새로운 스타일을 채택하고, 기존의 스타일은 더 이상 유행하지 않게 됩니다. 따라서, 유행의 변화는 사회적인 변화를 나타내는 지표로 볼 수 있습니다.


## 03. 추론- CARE

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model_care = Llama(
    model_path='./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=10000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.f

In [ ]:
prompt_care = '''
Context : "당신은 대입 논술 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다."
Action : "당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 새로운 제시문 세트로부터 당신에게 예시 제시문 세트와 예시 질문을 보여드리겠습니다. 이를 보고, 새로운 유사하게 제시문 세트로부터 질문을 생성하면 됩니다.질문을 생성하면 됩니다. 제시문 세트 안의 제시문 개수와, 질문의 개수는 정해져 있지 않습니다. "

Example : "예시 제시문 세트는 3개의 제시문으로 이뤄져 있습니다.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'

위의 예시 제시문 세트로부터 생성된 질문입니다.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'
"

Result : "이처럼 질문들은 제시문의 내용을 참고하여, 이를 요약하거나 이들로부터 생각해볼 수 있는 내용들을 질문합니다. 이때 각 제시문 내용간의 관계를 고려해, 이를 물어보는 문제를 출제해야 합니다. 당신은 이와 유사하게 새로운 제시문으로부터 질문을 생성하면 됩니다."
'''

In [ ]:
test_text_care = '''다음 3개의 제시문을 보고 질문을 생성해주세요.
질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다. [새로운 제시문]으로부터 질문을 생성하는 것입니다.
질문만 생성해주세요. 다른 불필요한 문장은 삭제해 주세요. 답변은 생성하지 않습니다.

생성 형식은
["질문 1"
"질문 2"
"질문 3"] 으로 해야 합니다.
질문을 3개 생성해주세요.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다.
이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다.
유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다.
그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다.
젊은 세대는 ‘반모’(반말모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
'''

In [ ]:


messages = [
    {"role": "system", "content": f"{prompt_care}"},
    {"role": "user", "content": f"{test_text_care}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":5000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.6,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model_care(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =     546.35 ms
llama_print_timings:      sample time =     199.41 ms /    62 runs   (    3.22 ms per token,   310.92 tokens per second)
llama_print_timings: prompt eval time =    1055.72 ms /   810 tokens (    1.30 ms per token,   767.25 tokens per second)
llama_print_timings:        eval time =    1779.16 ms /    61 runs   (   29.17 ms per token,    34.29 tokens per second)
llama_print_timings:       total time =    3194.70 ms /   871 tokens


질문:
1. 유행은 누구의 주도적인 역할을 통해 형성되는지 설명해주세요.
2. 어떤 요인들이 유행을 형성하는 데 중요한 역할을 하는지 설명하시오.
3. 왜 사람들은 유행에 민감하고, 새로운 변화에 쉽게 적응하는지 설명해주세요.


# 5.31 시도

## 03. 추론- RTF

In [ ]:
PROMPT = '''Role : "You are a Korean teacher who runs a Korean college essay interview academy"
Task : "Your main task is to generate example questions from the set of example statements. The number of statements in the set and the number of questions are not determined.
Let me show you an example set of statements and an example question, which you can look at, and generate a question from a new set of statements similarly.
It's just creating questions. The Questions must be generated in Korean, and concise in one or two sentences"

Format :
"First, I'll give you a set of statements as example. This set consists of 3 statements. All of statements are written in Korean.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'


Next, let me show you an example question: this is generated from the above set of example statements.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'

As such, example questions refer to the contents of the example statements and ask questions that can be summarized or thought about from them. At this time, it is necessary to consider the relationship between the contents of each presentation and ask questions that ask this."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고 질문을 생성해주세요. [새로운 제시문]으로부터 질문을 생성하는 것입니다.
질문만 생성해주세요. 다른 불필요한 문장은 삭제해 주세요.
생성 형식은
["질문 1"
"질문 2"
"질문 3"]
으로 부탁드립니다.
질문을 3개 생성해주세요. 질문만 생성하면 됩니다. 질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다. 답변은 필요 없습니다.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에
충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이
행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을
얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는
경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상
존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을
기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한
유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이
모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은
결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고
동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를
닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과
동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는
선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상
존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에
사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀
있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인,
프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운
것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에
확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서
변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을
하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령, ‘엄마, 아빠’와 같은 말은
어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적
행동 양식이 반영된 것이라고 할 수 있다. 이런 ‘연령 단계’에 의한 언어 차이 외에도,
기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도
있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를
쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말
모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와
신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는
젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  

In [ ]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =     539.71 ms
llama_print_timings:      sample time =     258.62 ms /   109 runs   (    2.37 ms per token,   421.47 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    3303.24 ms /   109 runs   (   30.30 ms per token,    33.00 tokens per second)
llama_print_timings:       total time =    3866.21 ms /   109 tokens


Here are three sample questions which can be generated from the above new set of example statements.

[질문 1]
유행에 대한 새로운 경향이 나타나는 경우, 어떤 원인들이 이를 이끌어나가는지 설명해주세요?

[질문 2]
최근에는 다양한 유행이 나타나면서 사람들 사이에서 어떤 요소가 중요한 역할을 하는지 설명해주세요.

[질문 3]
어떤 분야에서 유행이 생겨난다면, 해당 분야에서 유행을 따르는 사람들의 행동 양식은 어떻게 형성되는지 설명해주세요.


### 새로운 제시문에 접목시켜보기

In [ ]:
PROMPT_2 = '''Role : "You are a Korean teacher who runs a Korean college essay interview academy"
Task : "Your main task is to generate example questions from the set of new statements.
Let me show you an example set of statements and an example question, which you can look at, and generate a question from a new set of statements similarly.
This set of presentations contains content on one common topic. You should generate questions related to this common topic.
"

Format :
"First, I'll give you a set of statements as example. This set consists of 3 statements. All of statements are written in Korean.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'


Next, let me show you an example question: this is generated from the above set of example statements.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'

As such, example questions refer to the contents of the example statements and ask questions that can be summarized or thought about from them.
It's just creating questions. The Questions must be generated in Korean, and concise in one or two sentences
At this time, it is necessary to consider the relationship between the contents of each presentation and ask questions that ask this."
'''

In [ ]:
test_text_2 = '''다음 3개의 제시문을 보고 질문을 생성해주세요. 이때 유의사항이 몇 가지 있습니다.
첫 번째, [새로운 제시문]으로부터 질문을 생성하는 것입니다. 질문만 생성하고, 다른 불필요한 문장은 삭제해 주세요. 존대어를 사용하지 마세요.
두 번째, 생성 형식은
'
[질문 1]
"첫 번째 질문"
[질문 2]
"두 번째 질문"
[질문 3]
"세 번째 질문"
'
으로 부탁드립니다.
세 번째, 질문을 3개 생성해주세요. 질문은 한국어로, 1~2문장의 간결한 질문이 필요합니다. 답변은 필요 없습니다.
마지막으로, 질문은 진중한 어투로 제공되어야 합니다.

그렇다면 이제 제시문을 드리겠습니다.

[새로운 제시문 (가)]
진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다.
역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, ‘사실’을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다.
이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[새로운 제시문 (나)]
크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다.
유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다.
브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다.
순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다.
브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향
이었다.

[새로운 제시문 (다)]
대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대의 결론을 내렸다.
그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다.
골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다.
그의 명제는 단순하고 명확하다.
“독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.”
골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다.
그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
                '''

In [ ]:
messages = [
    {"role": "system", "content": f"{PROMPT_2}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =     790.26 ms
llama_print_timings:      sample time =     340.12 ms /   126 runs   (    2.70 ms per token,   370.45 tokens per second)
llama_print_timings: prompt eval time =    2316.05 ms /  2054 tokens (    1.13 ms per token,   886.86 tokens per second)
llama_print_timings:        eval time =    3695.48 ms /   125 runs   (   29.56 ms per token,    33.83 tokens per second)
llama_print_timings:       total time =    6551.85 ms /  2179 tokens


[질문 1]
유행에 대한 주제와 그의 형태에 대해 설명해주세요. 어떤 원인들이 유행을 형성하는 것이죠?

[질문 2]
유행이 모든 사회적 층을 관통하는 것이 아니라 일부 사람들 사이에서만 유행하는 경우가 있을 수 있습니다. 어떤 사회적 요인 때문에 일부 사람들 사이에서 유행이 형성되는 것일까요? 예를 들어 설명해주세요.

[질문 3]
유행은 왜 항상 변화할까요? 어떤 요인들이 유행의 변화를 일으킵니다?
유행이 멈추는 이유는 무엇인가요?


# 새로운 시도

### for 문으로 질문 독립적으로 생성

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
PROMPT = '''Role : "You are a Korean teacher who runs a Korean college essay interview academy"
Task : "Your main task is to generate example questions from the set of example statements.
Let me show you an example set of statements and an example question, which you can look at, and generate a question from a new set of statements similarly.
It's just creating questions. The Questions must be generated in Korean, and concise in one or two sentences"

Format :
"First, I'll give you a set of statements as example. This set consists of 3 statements. All of statements are written in Korean.

'
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가?
칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다.
기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다.
지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다.
그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다.
시체의 손가락에는 금반지가 끼워져 있었다.
기게스는 그 반지를 빼 들고 밖으로 나왔다.
그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다.
이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다.
그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다.
혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.
그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.
이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다.
대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다.
권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다.
대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다.
무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다.
그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다.
과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다.
그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다.
하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다.
이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다.
도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다.
왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다.
따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'

Next, let me show you an example question: this is generated from the above set of example statements.

[예시 질문]
'제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.'

As such, example questions refer to the contents of the example statements and ask questions that can be summarized or thought about from them. At this time, it is necessary to consider the relationship between the contents of each presentation and ask questions that ask this."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변을 생성해주세요. [새로운 제시문]으로부터 질문을 생성하는 것입니다. 다른 불필요한 문장은 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장의 간결한 질문이어야 합니다.
형식은 다음과 같습니다.

"질문"
"답변"

이제 새로운 제시문 세트를 보여드리겠습니다.

[새로운 제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에
충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이
행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을
얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는
경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상
존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을
기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한
유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이
모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은
결코 현재 상태에 머물지 않으며 부단히 진행된다.

[새로운 제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고
동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를
닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과
동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는
선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상
존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에
사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀
있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인,
프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운
것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에
확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서
변화하였다.

[새로운 제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을
하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령, ‘엄마, 아빠’와 같은 말은
어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적
행동 양식이 반영된 것이라고 할 수 있다. 이런 ‘연령 단계’에 의한 언어 차이 외에도,
기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도
있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를
쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말
모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와
신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는
젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
for i in range(3) :
  model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

  tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
  model = Llama(
      model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
      n_ctx=5000,
      n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
  )

  messages = [
      {"role": "system", "content": f"{PROMPT}"},
      {"role": "user", "content": f"{test_text}"}
      ]

  prompt = tokenizer.apply_chat_template(
      messages,
      tokenize = False,
      add_generation_prompt=True, verbose = 0
  )

  generation_kwargs = {
      "max_tokens":500,
      "stop":["<|eot_id|>"],
      "top_k" : 50,
      "top_p":0.9,
      "temperature":0.1,
      "echo":True, # Echo the prompt in the output
  }

  resonse_msg = model(prompt, **generation_kwargs)
  print(resonse_msg['choices'][0]['text'][len(prompt):])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[질문 1]
유행에 대한 다양한 관점이 포함되어 있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.

[답변 1]
제시문 11239년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두 각각의 독특한 방식으로 차려입기를 원했기 때문이다. 
유행이 전체로 지배하게 되면, 즉 처음에는 몇몇 사람들이 주도했다고 해서 모두가 따라 하게 되면, 그것이 옷이나 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 곧 소멸된다.
유행의 특성은 변화를 끊임없이 따르는 것이다. 예를 들어, 근대 유럽에서 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 이탈리아, 스페인, 프랑스가 이끄는 유행을 다른 국가의 상류층 사람들이 앞다냥으로 모방했다. 이들은 새로이 생겨난 것에 대해 무조건적으로 뒤쫓아가려고 했으며, 가장 최근에 생겨난 변화를 받아들이고자 했다.
따라서, 유행은 어떤 시기에는 유행이 되는 것이 다음에는 유행이 되지 않는 경향이 있다. 이는 사회적인 관심사와 사람들의 욕구가 변함에 따라 유행이 변화하기 때문이다.

[질문 2]
피렌체에서 남성 복장의 뚜렷하고 뚜렷한 유행이 없었던 이유는 모두 각각의 독특한 방식으로 차려입기를 원했기 때문입니다. 
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주체를 만들었고, 그 후에 모두 따라 하게 됩니다. 따라서, 유행은 사람들마다 서로 다른 모습을 가지도록 해줍니다.

[답변 2]
네, 맞습니다. 유행이 모든 것을 통제하고 지배하게 되면, 사람들은 자신의 개성을 표현할 수 없게 됩니다. 예를 들어, 피렌체에서 남성 복장의 뚜렷하고 뚜렷한 유행이 없었던 시기에는, 사람들은 각자의 취향과 개성에 따라 옷을 차려입었기 때문에 서로 다른 모습을 보였습니다. 
하지만, 유행이 모든 것을 통제하고 지배하게 되면, 사람들은 유행에 따라 옷을 차려입기 때문에 서로 같은 모습을 가지게 됩니다. 따라서, 유행이 모든 것을 지배하면 다양성과 개성이 감소하게 됩니다.

[질문 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[질문 1]
유행에 대한 다양한 관점이 포함되어 있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.

[답변 1]
제시문 (가), (나), (다)는 모두 유행에 대한 다양한 측면을 다루고 있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점은, 둘 다 유행이 인간의 행동과 심리적 욕구, 그리고 사회적 압력에 의해 형성된다는 점입니다. 반면에 제시문 (가)와 (나)의 차이점은, 제시문 (가)는 유행이 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시키는 매우 독특한 현상이라고 말하고, 제시문 (나)는 유행이 새로운 것을 숭배하고 동시대 사람들을 모방하는 경향이 있다는 것을 말합니다. 

제시문 (다)와 (na)의 공통점은, 둘 다 언어 사용과 관련된 유행이 있으며, 세대 변화에 따라 유행어와 신조어가 변화한다는 점입니다. 제시문 (다)는 특히 언어의 다양성과 유행어에 대한 관심을 강조하며, 제시문 (나)는 젊은 세대가 새로운 변화를 쉽게 받아들여 유행에 민감한 경향이 있다는 것을 설명합니다.

따라서, 제시문 (가), (나), (다)는 모두 유행을 다양한 측면에서 바라보며, 공통점과 차이점을 통해 그들의 특징과 변동을 이해할 수 있습니다.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[질문 1]
유행에 대한 다양한 관점이 포함되어 있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.

[답변 1]
제시문 (가), (나), (다)는 모두 유행에 대한 다양한 측면을 다루고 있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점은, 둘 다 유행이 인간의 행동과 관련이 있으며, 개인의 자유로운 선택과 사회적 압력에 의해 영향을 받는다는 것이다. 반면에 제시문 (가)와 (나)의 차이점은, 제시문 (가)는 유행이 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시키는 매우 독특한 현상이라고 말하고, 제시문 (나)는 유행이 새로운 것을 숭배하고 동시대 사람들을 모방하는 경향이 있다는 것이다. 

제시문 (다)와 (na)의 공통점은, 둘 다 언어 사용과 관련된 내용이 포함되어 있으며, 세대 변화에 따른 언어의 변화가 있다는 것이다. 제시문 (다)는 사람들이 사회적 존재이기 때문에 역할에 따라 행동양식이 달라지며, 이를 언어 사용에도 영향을 미친다고 말하고, 제시문 (나)는 젊은 세대가 새로운 변화를 쉽게 받아들인다는 점에서 세대 변화에 따른 언어 변화의 가능성을 언급하고 있다.


### 질문 형식의 예시를 여러 개 보여주기

In [3]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
PROMPT = '''Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성하면 됩니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가) 싱가포르의 ‘태형’과 (나) 흥보전의 ‘매품’을 서로 비교하시오."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 총 3개 생성해주세요. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"]
["답변 1"]

["질문 2"]
["답변 2"]

["질문 3"]
["답변 3"]
으로 부탁드립니다.

질문과 답변 세트를 3개 생성해주세요.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
[제시문 (가)]의 내용을 바탕으로, 어떤 이유로 사람들은 유행을 따르고 싶어하는 심리적 동기를 설명해주세요.

답변 1:
유행은 사람들이 자신들 주변의 다른 사람들과 일치하고 싶어하는 욕구와, 그들만의 개성을 표현하고자 하는 욕구가 복합적으로 얽혀 있는 것입니다. 따라서, 사람들은 상대들에게 인정과 안도감을 느끼기 위해 유행을 따르고 싶어합니다. 이는 그들이 주변 사람들에게 인정받고, 그들만의 스타일이 받아들여지고, 그들만의 개성이 존중받는 것을 의미합니다.

질문 2:
[제시문 (나)]의 내용을 바탕으로, 어떤 이유로 사람들은 옛 것들을 따라가고, 동시대 사람들을 모방하려는 심리적 동기를 설명해주세요.

답변 2:
옛것을 따르는 경향은 사람들의 변화를 평가절하하고, 현재의 규범을 중요시 하려는 경향이 함께 동반됩니다. 사람들은 과거를 존경하지 않고, 지금은 지금이 가장 중요한 것으로 여겨집니다. 또한, 사람들은 같은 시대를 사는 사람들과 연결하고자 하며, 동시대 사람들을 모방하려는 경향을 가지고 있습니다. 이는 사람들이 현재와 미래에 대한 불안감을 해소하고, 안정감을 느끼는 것을 의미합니다.

질문 3:
[제시문 (다)]의 내용을 바탕으로, 어떤 이유로 사람들은 언어 사용에서도 세대 차이와 연관이 있으며, 이를 어떻게 설명할 수 있을까요?

답변 3:
언어 사용에서도 세대 차이와 연관이 있습니다. 이는 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는데, 예를 들어, '엄마, 아빠'와 같은 말은 어린이들의 말이고, '자네, 댁'과 같은 단어는 어른들의 말이라는 것이 그러한 예입니다. 또한, 새로운 세대가 언어를 만들고 유행어를 만들기도 하는데, 이는 또 다른 세대 차이 중 하나인 젊은 세대의 특성을 대변합니다. 따라서, 언어 사용에서 세대 차이와 연관이 있는 것은 그들이 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는 것입니다.


### 건님의 instruction과 결합

In [12]:
qa_list = []

In [30]:
PROMPT = '''
Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성합니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [31]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 3개 생성해야 합니다. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"
"질문 2"
"질문 3"
"답변 1"
"답변 2"
"답변 3"]
으로 해야 합니다.

질문은 반드시 명령조로 생성되어야 합니다.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [32]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[질문 1]
제시문 (가)를 기반으로 질문을 만들면 다음과 같습니다.
가) 주어진 시기 동안 유행이 변화한 경우, 어떤 요인들이 그 변화를 가져왔을까요?
나) 전통적인 가치에 대한 반항과 현대 사회의 변화에 대한 수용성이 강한 시대에서는 유행이 어떻게 형성될 수 있을까요?
다) 어떤 집단은 옛 것들에 집착하고, 다른 사람들은 새로운 것에 열중하는 경우, 왜 이렇게 나타나는 것이일까요?

[질문 2]
제시문 (나)를 기반으로 질문을 만들면 다음과 같습니다.
가) 오래된 관습을 따르는 경향이 있는 문화에서는 어떠한 것들이 유행이 될 수 있을까요?
나) 어떤 사회에서는 변화에 대한 애정과 동시대인들에 대한 모방이 강력한 영향을 미치는지 분석해보세요.
다) 어떤 문화에서는 사람들이 가장 좋아하는 것, 즉 유행에 대해 어떻게 생각하느냐를 살펴봅시다.

[질문 3]
제시문 (다)를 기반으로 질문을 만들면 다음과 같습니다.
가) 언어 사용에서 어떠한 요인들이 유행이 되는 것을 설명하는 역할을 할 수 있을까요?
나) 어떤 사회에서는 청소년층과 어른들의 언어 사용할 방식이 다르게 형성되는지 분석해보세요.
다) 왜 사람들은 항상 새로운 유행에 관심을 가지며, 어떤 요인들이 유행의 변화가 발생하는지를 설명해주세요.


In [33]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

[질문 1]
유행은 어떤 심리적 원리와 사회적인 원리 때문에 생겨나는 것인가?

[답안 1]
유행은 크게 세 가지 심리적 원리와 사회적 원리로 설명될 수 있습니다. 첫째, 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 만족시키기 위해 나타납니다. 둘째, 다른 사람들과 구별되는 만족감을 얻으려고 하는 심리 때문입니다. 셋째, 변화를 수용할 수 있는 문화적인 요소 때문이죠. 이는 1390년경 피렌체에서 남성 복장의 뚜렷 유행이 나타나 이전과 이후에는 존재하지 않았던 것을 예시 들 수 있습니다.

[질문 2]
옛것을 따르는 습관은 어떤 원리와 시기에 가장 많이 나타났을까요?

[답안 2]
옛것을 따라가는 습관은 변화에 대한 애정과 동시대인들에 대한 모방 본능이 결합된 것으로 설명할 수 있습니다. 이 원리는 선조를 따르는 관습의 시대에 지배했던 것입니다. 지금은 주위 사람들을 닮으려 있는 경향이 유행의 시대를 특징 짓습니다. 이러한 원리들은 옛 것을 평가 절하하고 현재의 규범을 중요시 하려는 경향이 함께 동반됩니다. 

[질문 3]
언어는 어떤 사회적, 심리적 요인에 의해 변화할 수 있는지 설명해주세요.

[답안 3]
언어는 사회적인 존재이기 때문에 위치에 따라 그 행동양식이 달라지게 됩니다. 따라서, 언어 사용 역시 그러한 사회적 행동 양식에서 영향을 받게 됩니다. 예를 들어, '엄마, 아빠'와 같은 말은 어린이들의 말이고, '자네, 댁'과 같은 단어는 어른들의 말이라는 것은 나이에 따른 사회적 행동 양식이 반영된 것입니다. 또한, 젊은 세대는 새로운 변화에 민감하고, 새로운 변화를 쉽게 받아들이는 경향이 있는데, 이러한 특성에서는 '반모', '인싸'와 같은 유행어와 신조어가 만들어지는 것입니다. 이것은 세대 변화에 따른 언어의 유행 현상으로 이해할 수 있습니다.
**************************************************
[질문 1]
유행은 모든 연령대의 사람들에게 어떤 영향을 미치는가요?

[답변 1]
유행은 주로 청소년층에

### 영문입력

In [13]:
qa_list = []

In [43]:
PROMPT = '''
Role: "You are a teacher who runs a college entrance essay interview academy in South Korea."
Task: "Your main job is to generate example questions from a set of example statements.
I'm going to show you an example question. When you're given a new set of presentations, you figure out what they're about and then you create a similar set of questions.
At this time, the questions are to identify the relationship between the presentations and ask them."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [44]:
test_text = '''You need to look at the following 3 presentations and create 3 sets of questions and answers for them. Please delete all other unnecessary sentences.
All questions and answers should be in Korean, and concise, consisting of 1-2 sentences.
Each question should have a different form and content that should not overlap.
Please create three questions and three answers. As I said, when generating questions, the content should not imply or suggest the answers. When creating questions, it's important to consider the relationships between the contents of each prompt.
The question must be a commanding tone, and without honorifics.
Each question and answer should be related to all prompts.
Questions should be concise, and should not quote prompt.

The generation format is as follows:
["Question 1"]
"Question 2"
"Question 3"
"Answer 1"
"Answer 2"
[Answer 3]

Now, let me show you a new set of presentations.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [45]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.8,
    "temperature":0.3,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Here are three sets of questions and answers based on the three presents above.

Set 1:
- 질문 1: 유행은 누구의 욕구가 충족되는 현상인가?
답변 1: 유행은 주도적으로 유행을 만드는 사람들의 욕구가 충족되는 현상이다.
- 질문 2: 왜 유행이 중요한 역할을 하는가에 대해 설명해주세요.
답변 2: 유행은 사람들이 자신들 속에 있는 개성을 표현하고, 그들끼리 서로 다른 존재임을 확인하고자 하는 욕구를 충족시켜주는 역할을 한다.
- 질문 3: 어떤 조건에서 아래와 같은 말이 성립할 수 있을까요? "모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다."
답변 3: 모든 개인이 제각각 특수한 되고자 하는 욕구가 결여되는 경우이거나, 혹은 모방을 기피하는 집단의 경우이다.

Set 2:
- 질문 1: 어떤 원리로 인해 옛것을 따라가는 경향이 유행의 시기를 지배한다면?
답변 1: 옛것을 따라가는 현상은 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끄는 원리이다.
- 질문 2: 왜 대중들이 새로운 것을 선호하느냐에 대해 설명해주세요.
답변 2: 대중들은 새롭고 혁신적인 것에 대한 열정으로 가득 차 있으며, 이를 통해 자신들의 삶에서 변화를 바라는 욕구가 충족되기 때문이다.
- 질문 3: 어떤 시대적 상황 속에서 사람들은 옛 것을 따라갔는지에 대해 설명해주세요.
답변 3: 근대 유럽의 상류사회에서는 사람들이 새로운 것, 즉 최신 발명품과 이국적 문물에 대한 열정에 사로잡혀 있었고, 이를 선도하는 유행을 다른 국가의 상류층 사람들이 앞다 그랈다.

Set 3:
- 질문 1: 사회적인 존재로서의 역할은 어떤 영향을 미칠까?
답변 1: 사회적인 존재로서의 역할은 행동양식 중 하나가 언어 사용이다. '예를 들어,  ‘엄마, 아빠’와 같은 말은 어린이들의 말이고,  ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 영향을

In [46]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

[질문 1]
제시문 (가)의 내용을 기반으로, 어떤 사람들은 유행을 따른다는 이유가 무엇일까요? 

답변 1
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려고 하는 심리적인 이유와, 다른 사람들로부터 인정받고 동화되기 위한 만족감을 얻기 위해서이다. 

[질문 2]
제시문 (나)의 내용을 기반으로, 어떤 사람들은 왜 새로운 것을 숭배하고 동시대 사람들을 모방하느냐? 

답변 2
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 주요한 원리이다. 이 원칙들은 선조의 유산을 평가절하하고 현재의 규범을 중요시하려는 성향이 함께 동반된다. 

[질문 3]
제시문 (다)의 내용을 기반으로, 왜 사람들은 시대에 따라 다른 언어를 사용하느냐? 

답변 3
사람은 사회적인 존재이기 때문에 신분이나 역할에 따라 행동양식이 달라지며, 그 행동양식 중 하나가 언어 사용이다. 따라서, 세대 변화나 사회적 변화에 따라 사용되는 언어도 변화하게 된다. 특히, 젊은 세대는 새로운 유행어와 신조어를 만들어내며 유행에 민감하고 변화를 쉽게 받아들이는 경향이 때문이다.
**************************************************
[질문 1]
제시문 (가)의 내용을 바탕으로, 유행이 왜 모든 개인을 유혹하는 것인지 묻습니다.

[답변 1]
유행은 바로 다른 사람들과 나를 구분할 수 있는 좋은 방법이 되기 때문입니다. 모든 개인이 제각각 특수한 존재가 되고 싶어하는 욕구와 동시에 모방을 회피하고자 하는 심리적인 요인이 결합되어 있기 때문입니다. 

[질문 2]
제시문 (나)의 내용을 바탕으로, 왜 사람들은 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시기를 특징으로 만들게 되었을까요?

[답변 2]
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 두 가지 중요한 원리입니다. 이 원소들은 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 육안으로 결합됩니다. 

[질문 3]
제